In [9]:
import pandas as pd
import numpy as np

In [6]:
sales = pd.read_csv("kc_house_data.csv")
testing = pd.read_csv('wk3_kc_house_test_data.csv')
training = pd.read_csv('wk3_kc_house_train_data.csv')
validation = pd.read_csv('wk3_kc_house_valid_data.csv')

In [7]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [4]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [5]:
model_all.coef_

array([     0.        ,      0.        ,      0.        ,    134.43931396,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,  24750.00458561,      0.        ,
        61749.10309071,      0.        ,      0.        ,     -0.        ,
            0.        ])

In [10]:
l1_penalty_range = np.logspace(1, 7, num=13)

In [19]:
max_nonzero = 7
nonzero_list = []
for l1_penalty in l1_penalty_range:
    print l1_penalty
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features],training['price'])
    print ((model.predict(validation[all_features]) - validation['price'])**2).sum()
    nonzero_list.append(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))
print nonzero_list

10.0
3.982133273e+14
31.6227766017
3.99041900253e+14
100.0
4.29791604073e+14
316.227766017
4.63739831045e+14
1000.0
6.45898733634e+14
3162.27766017
1.22250685943e+15
10000.0
1.22250685943e+15
31622.7766017
1.22250685943e+15
100000.0
1.22250685943e+15
316227.766017
1.22250685943e+15
1000000.0
1.22250685943e+15
3162277.66017
1.22250685943e+15
10000000.0
1.22250685943e+15
[15, 15, 11, 6, 4, 1, 1, 1, 1, 1, 1, 1, 1]


# Compute test RSS on best model

In [13]:
model = linear_model.Lasso(alpha=10, normalize=True)
model.fit(training[all_features],training['price'])
print ((model.predict(testing[all_features]) - testing['price'])**2).sum()

9.84674025527e+13


In [14]:
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

In [27]:
l1_penalty_min = l1_penalty_range[2]
l1_penalty_max = l1_penalty_range[3]
print l1_penalty_min
print l1_penalty_max
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features],training['price'])
    if np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_) == max_nonzero:
        print l1_penalty
        print ((model.predict(validation[all_features]) - validation['price'])**2).sum()

100.0
316.227766017
156.902043689
4.40118263243e+14
168.282452426
4.41322817704e+14
179.662861164
4.42654859676e+14
191.043269902
4.44116566258e+14
202.42367864
4.45702397073e+14


In [25]:
l1_penalty_narrow_range = np.linspace(l1_penalty_min,l1_penalty_max,20)
model = linear_model.Lasso(alpha=l1_penalty_narrow_range[5], normalize=True)
model.fit(training[all_features],training['price'])
model.coef_

array([ -0.00000000e+00,  -0.00000000e+00,   1.04313641e+04,
         1.63441034e+02,   0.00000000e+00,  -0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         5.05817774e+05,   4.19474087e+04,   0.00000000e+00,
         1.16169193e+05,   0.00000000e+00,   0.00000000e+00,
        -2.60634807e+03,   0.00000000e+00])